<a href="https://colab.research.google.com/github/gilgarad/mldl4automation/blob/master/jupyter_notebooks/%EC%A7%81%EB%B0%A9%20%ED%85%8C%EC%8A%A4%ED%8A%B8%20-%20model5%20-%20k%20fold%2C%20subway%2C%20school%20feature%20added.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd /content/gdrive/My Drive/zigbang

/content/gdrive/My Drive/zigbang


In [3]:
import pandas as pd
from os.path import join
from os import listdir
from sklearn.model_selection import KFold

import tensorflow as tf
import keras.backend.tensorflow_backend as K
import numpy as np
import keras

Using TensorFlow backend.


In [0]:
rpath = '/content/gdrive/My Drive/zigbang'

In [19]:
print('Schools')
school_data = pd.read_csv(join(rpath, 'Schools.csv'))
school_data[:3]

Schools


,school_code,latitude,longitude,school_class,operation_type,highschool_type,gender,foundation_date,address_by_law
0,S000003511,37.490880,127.015082,elementary,national,NaN,both,1953.1.31,1165010800
1,S000003563,37.577782,127.002915,elementary,national,NaN,both,1946.8.22,1111016800
2,S010000737,37.481366,127.059055,elementary,public,NaN,both,1982.9.20,1168010300


In [18]:
print('Subways')
subway_data = pd.read_csv(join(rpath, 'Subways.csv'))
subway_data[:3]

Subways


,station_id,latitude,longitude,subway_line,address_by_law
0,1,37.555729,126.972145,"1,4,KJ,AP",1.114012e+09
1,2,37.565624,126.976936,"1,2",1.114017e+09
2,3,37.570169,126.983099,1,1.111013e+09


In [17]:
print('Train')
train_data = pd.read_csv(join(rpath, 'train.csv'))
train_data[:3]

Train


,key,apartment_id,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,...,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price
0,0,5584,1,200601,11~20,1999,47.43,6,37.585965,127.000231,...,4.0,individual,gas,91120,65.63,46,1.0,1.0,corridor,215000000
1,1,5584,1,200601,11~20,1999,44.37,8,37.585965,127.000231,...,4.0,individual,gas,91119,61.39,10,2.0,1.0,corridor,200000000
2,2,5059,1,200601,11~20,1992,54.70,8,37.580511,127.014016,...,9.0,individual,gas,8430,72.36,201,2.0,1.0,corridor,168000000


In [20]:
print('Test')
test_data = pd.read_csv(join(rpath, 'test.csv'))
test_data[:3]

Test


,key,apartment_id,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,...,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price
0,462533,3751,1,200912,21~31,1984,83.58,14,37.519926,127.052515,...,15.0,district,cogeneration,6648,107.19,2,3.0,1.0,corridor,0
1,764018,14029,1,201304,1~10,1968,107.17,10,37.572215,126.987643,...,8.0,individual,gas,46184,107.17,10,0.0,0.0,corridor,0
2,813528,12304,1,201309,21~30,1968,36.17,6,37.570067,127.010640,...,4.0,individual,gas,45540,39.66,58,2.0,1.0,stairway,0


# Initialization

In [0]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
keras.backend.set_session(sess)

# 전처리 파트

In [27]:
print('Finding None Numeric components, and Nan components')
non_numeric_components = list()
nan_include_components = list()

for k in train_data.keys():
    if isinstance(train_data[k][0], float) or str(train_data[k][0]).isdigit():
        isnumeric = True
    else:
        isnumeric = False
        non_numeric_components.append(k)
    include_nan = False
    if isnumeric and len(np.where(np.isnan(train_data[k]))[0]) > 0:
        nan_include_components.append(k)
        include_nan = True
        
    
    print(k, train_data[k][0], isnumeric, ' inculde NaN?', include_nan)
        
print('') 
print('Final Result')
print('None numeric components', non_numeric_components)
print('NAN include components', nan_include_components)

Finding None Numeric components, and Nan components
('key', 0, True, ' inculde NaN?', False)
('apartment_id', 5584, True, ' inculde NaN?', False)
('city', 1, True, ' inculde NaN?', False)
('transaction_year_month', 200601, True, ' inculde NaN?', False)
('transaction_date', '11~20', False, ' inculde NaN?', False)
('year_of_completion', 1999, True, ' inculde NaN?', False)
('exclusive_use_area', 47.43, True, ' inculde NaN?', False)
('floor', 6, True, ' inculde NaN?', False)
('latitude', 37.58596505640614, True, ' inculde NaN?', False)
('longitude', 127.00023096787315, True, ' inculde NaN?', False)
('address_by_law', 1111017100, True, ' inculde NaN?', False)
('total_parking_capacity_in_site', 163.0, True, ' inculde NaN?', True)
('total_household_count_in_sites', 136, True, ' inculde NaN?', False)
('apartment_building_count_in_sites', 1, True, ' inculde NaN?', False)
('tallest_building_in_sites', 8.0, True, ' inculde NaN?', True)
('lowest_building_in_sites', 4.0, True, ' inculde NaN?', True

In [28]:
print('전처리1: non numeric한 데이터를 변환하기 위한 딕셔너리 생성')
non_numeric_changer_dict = dict()
for k in non_numeric_components:
    train_data[k].fillna('nan', inplace=True)
    print(k)
    print(np.unique(train_data[k], return_counts=True))
    unique_list = np.unique(train_data[k], return_counts=True)[0]
    
    non_numeric_changer_dict2 = dict()
    for idx, label in enumerate(unique_list):
        non_numeric_changer_dict2[label] = idx
#         print('%s: %i' % (label, idx))
    non_numeric_changer_dict[k] = non_numeric_changer_dict2
    print('')

print(non_numeric_changer_dict)
print('#####')
    



전처리1: non numeric한 데이터를 변환하기 위한 딕셔너리 생성
transaction_date
(array(['11~20', '1~10', '21~28', '21~29', '21~30', '21~31'], dtype=object), array([538876, 513530,  31679,   9569, 171902, 335902]))

heat_type
(array(['central', 'district', 'individual', 'nan'], dtype=object), array([ 133749,  379822, 1085870,    2017]))

heat_fuel
(array(['-', 'cogeneration', 'gas', 'nan'], dtype=object), array([   8971,  378141, 1204679,    9667]))

front_door_structure
(array(['-', 'corridor', 'mixed', 'nan', 'stairway'], dtype=object), array([     21,  415209,   26085,   13892, 1146251]))

{'heat_fuel': {'cogeneration': 1, 'gas': 2, 'nan': 3, '-': 0}, 'transaction_date': {'21~31': 5, '21~30': 4, '21~28': 2, '21~29': 3, '1~10': 1, '11~20': 0}, 'heat_type': {'nan': 3, 'individual': 2, 'central': 0, 'district': 1}, 'front_door_structure': {'mixed': 2, 'nan': 3, 'stairway': 4, '-': 0, 'corridor': 1}}
#####


In [29]:
def replace_elements(data):
    print('전처리2: non numeric을 numeric으로 변경')
    for k, v in non_numeric_changer_dict.items():
        for k2, v2 in v.items():
            data[k][np.where(data[k]==k2)[0]] = v2

    print('전처리2: Numeric column 이지만 NaN 값을 가지고 있으면 0으로 변경')
    for k in nan_include_components:
        data[k].fillna(0, inplace=True)
        print(k)

        print('')
        
replace_elements(train_data)
replace_elements(test_data)

전처리2: non numeric을 numeric으로 변경


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


전처리2: Numeric column 이지만 NaN 값을 가지고 있으면 0으로 변경
total_parking_capacity_in_site

tallest_building_in_sites

lowest_building_in_sites

room_count

bathroom_count

전처리2: non numeric을 numeric으로 변경
전처리2: Numeric column 이지만 NaN 값을 가지고 있으면 0으로 변경
total_parking_capacity_in_site

tallest_building_in_sites

lowest_building_in_sites

room_count

bathroom_count



In [30]:
y_train = train_data['transaction_real_price'].values
# x_train = train_data.loc[:, train_data.columns != 'transaction_real_price'].values
x_train = train_data.loc[:, ~train_data.columns.isin(['transaction_real_price', 'key'])].values

print(x_train.shape)
print(y_train.shape)

(1601458, 23)
(1601458,)


In [31]:
y_test = test_data['transaction_real_price'].values
x_test = test_data.loc[:, ~test_data.columns.isin(['transaction_real_price', 'key'])].values

print(x_test.shape)
print(y_test.shape)

(3918, 23)
(3918,)


# Model Definition

In [0]:
from keras.models import Model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input, Conv2D, Flatten, MaxPooling2D, AveragePooling2D
from keras import optimizers
from sklearn.model_selection import train_test_split
from keras.callbacks import CSVLogger

# MLP

In [0]:
class MLP:
    def __init__(self):
        with K.tf.device('/gpu:0'):
            inputs = Input(shape=(23, ))
#             x = Dropout(0.5)(inputs)
#             m = BatchNormalization()(inputs)
            m = inputs
    
            num_neurons = 512
            keep_dropout_rate = 0.5
            x = Dense(num_neurons)(m)
            x = Activation("relu")(x)
            x = Dropout(keep_dropout_rate)(x)

#             x = Dense(num_neurons)(x)
#             x = Activation("relu")(x)
#             x = Dropout(keep_dropout_rate)(x)

#             x = Dense(num_neurons)(x)
#             x = Activation("relu")(x)
#             x = Dropout(keep_dropout_rate)(x)

#             x = Dense(num_neurons)(x)
#             x = Activation("relu")(x)
#             x = Dropout(keep_dropout_rate)(x)

#             x = Dense(num_neurons)(x)
#             x = Activation("relu")(x)
#             x = Dropout(keep_dropout_rate)(x)

            x = Dense(num_neurons)(x)
#             x = Activation("relu")(x)
#             x = Dropout(keep_dropout_rate)(x)

            m = Dense(1)(x)

            model = Model(inputs=inputs, outputs=[m])
        
        def root_mean_squared_error(y_true, y_pred):
            return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
        
#         adam = optimizers.Adam(lr=0.001)
        model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
        print(model.summary())
        self.model = model
        
    def train(self, x_train, y_train, epochs=50, batch_size=128, verbose=1):
        self.model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose, shuffle=True)
        
    def evaluate(self, x_test, y_test):
        score = self.model.evaluate(x_test, y_test, verbose=0)
        print("Test MSE is ", score)
        return score
    
    def predict(self, x_test, batch_size=1):
        _y = self.model.predict(x=x_test, batch_size=batch_size)
        return _y

In [0]:
mlp = MLP()
# x_train = input_data.reshape(input_data.shape[0], 1, input_data.shape[1], 1)
_x_train, _x_test, _y_train, _y_test = train_test_split(x_train, y_train, shuffle=True, test_size=0.1)
all_scores = list()
for i in range(500):
    print(i)
    mlp.train(x_train=_x_train, y_train=_y_train, epochs=1, batch_size=2048)
    all_scores.append(mlp.evaluate(x_test=_x_test, y_test=_y_test))
    print('')
    
for i in range(50):
    print(i)
    mlp.train(x_train=x_train, y_train=y_train, epochs=1, batch_size=2048)
    all_scores.append(mlp.evaluate(x_test=_x_test, y_test=_y_test))
    print('')

In [0]:
mlp_predicted = mlp.predict(x_test=x_test)

In [0]:
sub = pd.DataFrame()
sub['key'] = test_data['key']
sub['transaction_real_price'] = mlp_predicted
sub.to_csv('submission_model2_mlp.csv',index=False)

# CNN

In [0]:
class CNN:
    def __init__(self, gpu=0):
        with K.tf.device('/gpu:' + str(gpu)):
            inputs = Input(shape=(1, 23, 1))
            
            num_neurons = 512

            m = Conv2D(num_neurons, kernel_size=(1, 11), padding='valid')(inputs)
            m = BatchNormalization()(m)
            m = Activation('relu')(m)
            m = AveragePooling2D(pool_size=(1, 2), strides=(1, 1))(m)
            
            m = Conv2D(num_neurons, kernel_size=(1, 7), padding='valid')(m)
            m = BatchNormalization()(m)
            m = Activation('relu')(m)
            m = AveragePooling2D(pool_size=(1, 2), strides=(1, 1))(m)
            
            m = Conv2D(num_neurons, kernel_size=(1, 3), padding='valid')(m)
            m = BatchNormalization()(m)
            m = Activation('relu')(m)
            m = AveragePooling2D(pool_size=(1, 2), strides=(1, 1))(m)
            
#             m = Conv2D(num_neurons, kernel_size=(1, 3), padding='valid')(m)
#             m = BatchNormalization()(m)
#             m = Activation('relu')(m)
#             m = AveragePooling2D(pool_size=(1, 2), strides=(1, 1))(m)
            
#             m = Conv2D(num_neurons, kernel_size=(1, 3), padding='valid')(m)
#             m = BatchNormalization()(m)
#             m = Activation('relu')(m)
#             m = AveragePooling2D(pool_size=(1, 2), strides=(1, 1))(m)
            
            m = Flatten()(m)
            m = Dense(1000, activation='relu')(m)
            m = Dense(1)(m)

            model = Model(inputs=inputs, outputs=m)
            
        def root_mean_squared_error(y_true, y_pred):
            return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
        
        adam = optimizers.Adam(lr=0.001)        
        model.compile(loss='mean_squared_error', optimizer=adam, metrics=['accuracy'])
        print(model.summary())
        self.model = model
        
    def train(self, x_train, y_train, x_test, y_test, epochs=50, batch_size=128, verbose=1, csv_logger=None):
        if csv_logger is None:
            self.model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size, verbose=verbose,
                           shuffle=True)
        else:
            self.model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size, verbose=verbose,
                           shuffle=True, callbacks=[csv_logger])
        
    def evaluate(self, x_test, y_test, batch_size=1):
        score = self.model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)
        print("Test MSE is ", score)
        return score
    
    def predict(self, x_test, batch_size=1):
        _y = self.model.predict(x=x_test, batch_size=batch_size)
        return _y

In [0]:
# _x_train, _x_test, _y_train, _y_test = train_test_split(x_train, y_train, shuffle=True, test_size=0.1)
batch_size = 1024
epochs = 40

kf = KFold(n_splits=10)
kf.get_n_splits(x_train)

all_models = list()

csv_logger = CSVLogger(join(rpath, 'log.csv'), append=True, separator=';')

for idx, (train_index, test_index) in enumerate(kf.split(x_train)):
    print('########################################')
    print('K Fold:', idx)
    print('########################################')
    cnn = CNN(gpu=1)

    _x_train = x_train[train_index]
    _y_train = y_train[train_index]
    _x_test = x_train[test_index]
    _y_test = y_train[test_index]
    
    _x_train = _x_train.reshape(_x_train.shape[0], 1, _x_train.shape[1], 1)
    _x_test = _x_test.reshape(_x_test.shape[0], 1, _x_test.shape[1], 1)
    
    cnn.train(x_train=_x_train, y_train=_y_train, x_test=_x_test, y_test=_y_test, epochs=epochs, batch_size=batch_size, verbose=1,
             csv_logger=csv_logger)
    all_models.append(cnn)
#     all_scores.append(cnn.evaluate(x_test=_x_test, y_test=_y_test, batch_size=batch_size))

########################################
('K Fold:', 0)
########################################
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 23, 1)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 13, 512)        6144      
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 13, 512)        2048      
_________________________________________________________________
activation_1 (Activation)    (None, 1, 13, 512)        0         
_________________________________________________________________
average_pooling2d_1 (Average (None, 1, 12, 512)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 6, 512)         1835520   
_____________________________________________

In [0]:
_x_test = x_test.reshape(x_test.shape[0], 1, x_test.shape[1], 1)
cnn_predicted_all = list()
for cnn in all_models:
    cnn_predicted = cnn.predict(x_test=_x_test, batch_size=batch_size)
    cnn_predicted_all.append(cnn_predicted)
cnn_predicted_all = np.array(cnn_predicted_all)
cnn_predicted_final = np.mean(cnn_predicted_all, axis=0)

In [0]:
# print(a[0][0])
# print(np.mean([23720146, 31678532, 24474986, 26008190, 25262484, 21582632, 28799930, 29443314, 27432226, 36861810], axis=0))
# print(np.median([23720146, 31678532, 24474986, 26008190, 25262484, 21582632, 28799930, 29443314, 27432226, 36861810], axis=0))

In [0]:
sub = pd.DataFrame()
sub['key'] = test_data['key']
sub['transaction_real_price'] = cnn_predicted_final
sub.to_csv(join(rpath, 'submission_model5_cnn.csv'),index=False)